In [1]:
import os
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, String, Float, DateTime
import psycopg2

In [2]:
# Caminho para o arquivo de chave da conta de serviço - Lucas disponibilizou no Whatsapp do grupo
service_account_path = 'tfm-sa.json'

# Criar objeto de credenciais
credentials = service_account.Credentials.from_service_account_file(service_account_path)

In [3]:
project_id = 'perseverance-332400'
dataset_id = 'TFM'
table_id = 'ds_market'
full_table_id = f'{project_id}.{dataset_id}.{table_id}'
client = bigquery.Client(project='perseverance-332400', credentials=credentials)

In [4]:
user = os.environ.get('DB_USER')
password = os.environ.get('DB_PASSWORD')
host = os.environ.get('DB_HOST', 'localhost')
port = os.environ.get('DB_PORT', '5432')
database = os.environ.get('DB_NAME')
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

In [5]:
years = range(2011, 2017)  # From 2011 to 2016

In [6]:
def clean_data(df):
    # Renomear 'category_x' para 'category'
    df = df.rename(columns={'category_x': 'category'})

    # Remover a coluna 'category_y' se existir
    if 'category_y' in df.columns:
        df = df.drop('category_y', axis=1)

    # Remover a coluna 'd' se existir
    if 'd' in df.columns:
        df = df.drop('d', axis=1)

    # Preencher valores nulos na coluna 'event' com strings vazias
    if 'event' in df.columns:
        df['event'] = df['event'].fillna('')

    # Preencher valores nulos em 'sell_price' com zeros
    df['sell_price'] = df['sell_price'].fillna(0)

    # Converter 'sell_price' para float32 com 2 casas decimais
    df['sell_price'] = df['sell_price'].astype(float).round(2)
    df['sell_price'] = df['sell_price'].astype('float32')

    return df

In [7]:
dtype_mapping = {
    'id': String,
    'item': String,
    'category': String,
    'department': String,
    'store': String,
    'store_code': String,
    'region': String,
    'sales': Integer,
    'yearweek': Integer,
    'date': DateTime,
    'event': String,
    'sell_price': Float,
    'year': Integer
}

In [8]:

# Nome da tabela no PostgreSQL
table_name = 'ds_market'

for year in years:
    try:
        print(f"Processando dados para o ano {year}...")

        # Definir a consulta SQL para o ano atual
        query = f"""
        SELECT
          id,
          item,
          category_x,
          category_y,
          department,
          store,
          store_code,
          region,
          d,
          sales,
          yearweek,
          date,
          event,
          sell_price
        FROM
          `perseverance-332400.TFM.ds_market`
        WHERE
          EXTRACT(YEAR FROM date) = {year}
        """

        # Executar a consulta e obter o DataFrame
        df_year = client.query(query).to_dataframe()

        # Limpar os dados
        df_year = clean_data(df_year)

        # Definir o modo de inserção
        if_exists_mode = 'append' if year != 2011 else 'replace'

        # Escrever o DataFrame no PostgreSQL
        df_year.to_sql(
            table_name,
            engine,
            if_exists=if_exists_mode,
            index=False,
            method='multi',
            chunksize=10000,
            dtype=dtype_mapping
        )

        # Liberar memória
        del df_year
        import gc
        gc.collect()

    except Exception as e:
        print(f"Erro ao processar o ano {year}: {e}")

print("Processamento e salvamento concluídos.")


Processando dados para o ano 2011...


/home/rp/Desenvolvimento/DSMarket/venv/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Processando dados para o ano 2012...


/home/rp/Desenvolvimento/DSMarket/venv/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Processando dados para o ano 2013...


...

Processando dados para o ano 2015...


/home/rp/Desenvolvimento/DSMarket/venv/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Processando dados para o ano 2016...


/home/rp/Desenvolvimento/DSMarket/venv/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Processamento e salvamento concluídos.


In [9]:
df_check = pd.read_sql('SELECT * FROM ds_market LIMIT 5', engine)
print(df_check.head())

                          id                 item       category  \
0  HOME_&_GARDEN_2_352_NYC_2  HOME_&_GARDEN_2_352  HOME_&_GARDEN   
1  HOME_&_GARDEN_2_507_NYC_2  HOME_&_GARDEN_2_507  HOME_&_GARDEN   
2  HOME_&_GARDEN_2_476_NYC_2  HOME_&_GARDEN_2_476  HOME_&_GARDEN   
3  HOME_&_GARDEN_2_346_NYC_2  HOME_&_GARDEN_2_346  HOME_&_GARDEN   
4  HOME_&_GARDEN_2_489_NYC_2  HOME_&_GARDEN_2_489  HOME_&_GARDEN   

        department   store store_code    region  sales  yearweek       date  \
0  HOME_&_GARDEN_2  Harlem      NYC_2  New York      0    201134 2011-08-22   
1  HOME_&_GARDEN_2  Harlem      NYC_2  New York      1    201124 2011-06-15   
2  HOME_&_GARDEN_2  Harlem      NYC_2  New York      0    201149 2011-12-07   
3  HOME_&_GARDEN_2  Harlem      NYC_2  New York      0    201138 2011-09-19   
4  HOME_&_GARDEN_2  Harlem      NYC_2  New York      0    201107 2011-02-14   

  event  sell_price  
0              3.68  
1              8.71  
2              0.00  
3              0.00  
4     